In [ ]:
import os
import sys
from pathlib import Path
import logging
import subprocess
import tempfile
import numpy as np
from scipy.sparse import csr_array
import h5py
import matplotlib.pyplot as plt
from heavyhex_qft.triangular_z2 import TriangularZ2Lattice
from heavyhex_qft.plaquette_dual import PlaquetteDual
import skqd_z2lgt

# cuda_visible_devices = [1, 2, 4, 5, 6, 7]
CUDA_VISIBLE_DEVICES = list(range(1, 5))

logging.basicConfig(level=logging.WARNING)
LOG = logging.getLogger('skqd_z2lgt')
LOG.setLevel(logging.INFO)

SCRIPT_DIR = Path(skqd_z2lgt.__file__).parent / 'tasks'

In [ ]:
def read_skqd_result(filename, groupname):
    with h5py.File(filename, 'r') as source:
        group = source[groupname]
        dataset = group['sqd_states']
        num_bits = dataset.attrs['num_bits']
        sqd_states = np.unpackbits(dataset[()], axis=1)[:, :num_bits]
        data = group['ham_proj/data'][()]
        indices = group['ham_proj/indices'][()]
        indptr = group['ham_proj/indptr'][()]
        shape = (indptr.shape[0] - 1,) * 2
        ham_proj = csr_array((data, indices, indptr), shape=shape)
        energy = group['energy'][()]

    return sqd_states, ham_proj, energy

## Basic setup

In [ ]:
# output_filename = '/data/iiyama/2dz2/full_workflow_kawasaki_64plaq_K1.0_dt1overnp.h5'
output_filename = '/tmp/K1.0_dt1over64.h5'

# configuration = {
#     'lattice': '''
#   ^ ^ ^
#  * * * *
# * * * * *
#  * * * *
# * * * * *
#  * * * *
# * * * * *
#  * * * *
#   v v v
# ''',
#     'plaquette_energy': 0.8,
#     'delta_t': 0.5,
#     'num_steps': 5,
#     'shots': 100_000,
#     'basis_2q': 'cz',
#     'instance': 'ICEPP-dedicated-temp-prem-us',
#     'backend': 'ibm_pittsburgh'
# }
configuration = {
    'lattice': '''
*-*-*-*-*╷
 * * * * *
* * * * *╎
 * * * * *
* * * * *╎
 * * * * *
* * * * *╎
 * * * * *
*-*-*-*-*╵
''',
    'plaquette_energy': 1.,
    'delta_t': 1. / 64,
    'num_steps': 8,
    'basis_2q': 'rzz',
    'instance': 'ICEPP-dedicated',
    'backend': 'ibm_kawasaki'
}



In [ ]:
try:
    with h5py.File(output_filename, 'r') as source:
        for key, record in source.attrs.items():
            if (value := configuration.get(key)) is None:
                configuration[key] = record
            else:
                if ((isinstance(record, float) and not np.isclose(record, value))
                        or isinstance(record, (int, str)) and record != value):
                    raise ValueError(f'Wrong {key}!')

except FileNotFoundError:
    with h5py.File(output_filename, 'w-', libver='latest') as output_file:
        for key, value in configuration.items():
            output_file.attrs[key] = value

In [ ]:
lattice = TriangularZ2Lattice(configuration['lattice'])

dual_lattice = PlaquetteDual(lattice)
ising_hamiltonian = dual_lattice.make_hamiltonian(configuration['plaquette_energy'])

num_link = lattice.num_links
num_vtx = lattice.num_vertices
num_plaq = lattice.num_plaquettes
LOG.info('Number of links: %d, vertices: %d, plaquettes: %d', num_link, num_vtx, num_plaq)

## Compute the approximate ground-state energy

In [ ]:
with h5py.File(output_filename, 'r', swmr=True) as source:
    compute = 'dmrg' not in source

if compute:
    proc = subprocess.run(['python', SCRIPT_DIR / 'dmrg.py', output_filename],
                          capture_output=True, text=True, check=False)
    sys.stdout.write(proc.stdout)
    sys.stderr.write(proc.stderr)
    proc.check_returncode()

with h5py.File(output_filename, 'r', swmr=True) as source:
    dmrg_energy = source['dmrg/energy'][()]

print('DMRG energy:', dmrg_energy)

## Get the experiment data

In [ ]:
with h5py.File(output_filename, 'r', swmr=True) as source:
    data_group = source.get('data')
    if data_group is None:
        submit = compute = True
    else:
        submit = 'raw' not in data_group
        compute = 'vtx' not in data_group or 'plaq' not in data_group

if submit:
    cmd = ['python', SCRIPT_DIR / 'experiment.py', output_filename, '--shots', '100000']
    cmd += ['--job-id', 'd3mk71pfk6qs73e827bg']
    proc = subprocess.run(cmd, capture_output=True, text=True, check=False)
    sys.stdout.write(proc.stdout)
    sys.stderr.write(proc.stderr)
    proc.check_returncode()

if compute:
    proc = subprocess.run(['python', SCRIPT_DIR / 'data.py', output_filename],
                          capture_output=True, text=True, check=False)
    sys.stdout.write(proc.stdout)
    sys.stderr.write(proc.stderr)
    proc.check_returncode()

## Test the SKQD accuracy without configuration recovery

In [ ]:
with h5py.File(output_filename, 'r') as source:
    compute = 'skqd_raw' not in source

if compute:
    cmd = ['python', SCRIPT_DIR / 'skqd_raw.py', output_filename,
           '--gpu'] + list(map(str, CUDA_VISIBLE_DEVICES))
    proc = subprocess.run(cmd, capture_output=True, text=True, check=False)
    sys.stdout.write(proc.stdout)
    sys.stderr.write(proc.stderr)
    proc.check_returncode()

sqd_states_raw, ham_proj_raw, energy_raw = read_skqd_result(output_filename, 'skqd_raw')

LOG.info('Basis size: %d', sqd_states_raw.shape[0])
LOG.info('Matrix density %d', ham_proj_raw.data.shape[0])
LOG.info('Raw bitstrings energy: %f', energy_raw)

## Test SKQD with random plaquette flips

In [ ]:
num_exps = 1
sqd_states_rnd = {}
ham_proj_rnd = {}
energy_rnd = {}

compute_iexp = []
for iexp in range(num_exps):
    with h5py.File(output_filename, 'r') as source:
        if f'skqd_rnd_{iexp}' not in source:
            compute_iexp.append(iexp)

if compute_iexp:
    cmd = ['python', SCRIPT_DIR / 'skqd_random.py', output_filename]
    cmd += [f'{iexp}' for iexp in compute_iexp]
    cmd += ['--gpu'] + list(map(str, CUDA_VISIBLE_DEVICES))
    cmd += ['--num-gen', '3']
    proc = subprocess.run(cmd, capture_output=True, text=True, check=False)
    sys.stdout.write(proc.stdout)
    sys.stderr.write(proc.stderr)
    proc.check_returncode()

for iexp in range(num_exps):
    result = read_skqd_result(output_filename, f'skqd_rnd_{iexp}')
    sqd_states_rnd[iexp] = result[0]
    ham_proj_rnd[iexp] = result[1]
    energy_rnd[iexp] = result[2]

    LOG.info('Experiment %d basis size: %d', iexp, sqd_states_rnd[iexp].shape[0])
    LOG.info('Matrix density %d', ham_proj_rnd[iexp].data.shape[0])
    LOG.info('Energy %f', energy_rnd[iexp])

## Train CRBMs for configuration recovery

In [ ]:
procs = {}
for istep in range(configuration['num_steps']):
    groupname = f'crbm/step{istep}'
    with h5py.File(output_filename, 'r') as source:
        compute = f'step{istep}' not in source.get('crbm', {})

    if compute:
        with tempfile.NamedTemporaryFile() as tfile:
            out_name = tfile.name

        cmd = ['python', f'{SCRIPT_DIR}/train_crbm.py', output_filename, f'{istep}',
               '--gpu', f'{CUDA_VISIBLE_DEVICES[istep % len(CUDA_VISIBLE_DEVICES)]}',
               '--out', out_name, '--num-epochs', '100', '--rtol', '2.']
        procs[istep] = (
            subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True),
            out_name
        )

for istep, (proc, name) in procs.items():
    stdout, stderr = proc.communicate()
    sys.stdout.write(stdout)
    sys.stderr.write(stderr)
    if proc.returncode != 0:
        raise subprocess.SubprocessError()
    with h5py.File(output_filename, 'r+') as out:
        try:
            del out[f'crbm/step{istep}']
        except KeyError:
            pass

        with h5py.File(name, 'r') as source:
            source.copy(f'crbm/step{istep}', out.get('crbm') or out.create_group('crbm'))

    os.unlink(name)

In [ ]:
fig, axes = plt.subplots(4, np.ceil(configuration['num_steps'] / 4).astype(int), sharex=True)
fig.set_figheight(8.)

for istep in range(configuration['num_steps']):
    with h5py.File(output_filename, 'r') as source:
        train_loss = source[f'crbm/step{istep}/records/train_loss'][()]
        test_loss = source[f'crbm/step{istep}/records/test_loss'][()]

    evals_per_epoch = train_loss.shape[0] // test_loss.shape[0]

    ax = fig.axes[istep]
    ax.set_title(f'Step {istep}', fontsize=8)
    x = np.arange(len(train_loss)) / evals_per_epoch
    ax.plot(x, train_loss)
    x = np.arange(1, len(test_loss) + 1)
    ax.plot(x, test_loss)

fig.tight_layout()

## Test SKQD with configuration recovery

In [ ]:
with h5py.File(output_filename, 'r', swmr=True) as source:
    compute = 'skqd_rcv' not in source

if compute:
    cmd = ['python', SCRIPT_DIR / 'skqd_recovery.py', output_filename,
           '--gpu', f'{CUDA_VISIBLE_DEVICES[0]}',
           '--num-gen', '3', '--niter', '10',
           '--terminate', 'diff=0.05', 'dim=800000']
    proc = subprocess.run(cmd, capture_output=True, text=True)
    sys.stdout.write(proc.stdout)
    sys.stderr.write(proc.stderr)
    proc.check_returncode()

sqd_states_rcv, ham_proj_rcv, energy_rcv = read_skqd_result(output_filename, 'skqd_rcv')

LOG.info('Basis size: %d', sqd_states_rcv.shape[0])
LOG.info('Matrix density %d', ham_proj_rcv.data.shape[0])
LOG.info('Recovered bitstrings energy: %f', energy_rcv)

In [ ]:
dmrg_energy, energy_raw, energy_rnd, energy_rcv